In [229]:
import pandas as pd
import numpy as np
excel_path=r"C:\Users\ASUS\OneDrive\文档\明日方舟.xlsx"

In [230]:
name_list=pd.read_excel(excel_path,sheet_name='干员信息')['干员']

In [231]:
import requests
wiki_url='http://prts.wiki/w/'
operator_url=''

In [232]:
def findSubStr(substr, str, i):#查找子字符串第i次出现的位置
    count = 0
    while i > 0:
        index = str.find(substr)
        if index == -1:
            return -1
        else:
            str = str[index+1:]   #第一次出现的位置截止后的字符串
            i -= 1
            count = count + index + 1   #字符串位置数累加
    return count - 1

In [233]:
df=pd.DataFrame(columns = ['代号','生命上限', '攻击','防御','法术抗性','阻挡数','再部署时间','初始部署费用','攻击间隔','技能一初始','技能一消耗','技能一持续','技能二初始','技能二消耗','技能二持续'])

In [234]:
name='陈'# for name in name_list:
df['代号']=name
Attribute_list=['阻挡数','攻击间隔','生命上限','攻击','防御','法术抗性']
attribute_list=[]
for Attribute in Attribute_list:
    Attribute_str='<th>{}\n</th>'.format(Attribute)
    operator_url=wiki_url+name
    r = requests.get(operator_url)
    #大标题定位
    r_pos=r.text[r.text.find(Attribute_str):]
    #属性值
    attribute=int(r_pos[findSubStr('<td>',r_pos,3)+len('<td>'):].split('\n')[0])
    #对应信赖加成
    attribute_reliance=r_pos[findSubStr('<td>',r_pos,5)+len('<td>'):].split('\n')[0]
    if attribute_reliance=='':
        attribute_reliance=0
    else:
        attribute_reliance=int(attribute_reliance)
    attribute+=attribute_reliance
    attribute_list.append(attribute)
    #返回结果
    attribute,attribute_reliance


df=df.append(dict(zip(Attribute_list,attribute_list)), ignore_index=True)



In [235]:
skill1='<p><b>技能1（精英0开放）</b>'
r_pos=r.text[r.text.find(skill1):]
temp=[]
temp.append(r_pos[findSubStr('<td>',r_pos,26)+len('<td>'):].split('\n')[0])#初始
temp.append(r_pos[findSubStr('<td>',r_pos,27)+len('<td>'):].split('\n')[0])#消耗
temp.append(r_pos[findSubStr('<td>',r_pos,28)+len('<td>'):].split('\n')[0])#持续
for x in temp:
    if x=='':
        temp[temp.index(x)]=0
df['技能一初始']=temp[0]
df['技能一消耗']=temp[1]
df['技能一持续']=temp[2]

In [236]:
skill2='<p><b>技能2（精英1开放）</b>'
r_pos=r.text[r.text.find(skill2):]
temp=[]
temp.append(r_pos[findSubStr('<td>',r_pos,26)+len('<td>'):].split('\n')[0])#初始
temp.append(r_pos[findSubStr('<td>',r_pos,27)+len('<td>'):].split('\n')[0])#消耗
temp.append(r_pos[findSubStr('<td>',r_pos,28)+len('<td>'):].split('\n')[0])#持续
for x in temp:
    if x=='':
        temp[temp.index(x)]=0
df['技能二初始']=temp[0]
df['技能二消耗']=temp[1]
df['技能二持续']=temp[2]

In [237]:
df

,代号,生命上限,攻击,防御,法术抗性,阻挡数,再部署时间,初始部署费用,攻击间隔,技能一初始,技能一消耗,技能一持续,技能二初始,技能二消耗,技能二持续
0,NaN,2188.0,519.0,338.0,0.0,3417.0,NaN,NaN,4564.0,0,5,0,14,25,0


In [238]:
string='<td width="33.4%">'
r_pos=r.text[r.text.find(string):]
df['再部署时间']=r_pos[findSubStr('>',r_pos,1)+len('>'):].split('s')[0]
df['初始部署费用']=r_pos[findSubStr('>',r_pos,5)+len('>'):].split('→')[1]

In [239]:
df

,代号,生命上限,攻击,防御,法术抗性,阻挡数,再部署时间,初始部署费用,攻击间隔,技能一初始,技能一消耗,技能一持续,技能二初始,技能二消耗,技能二持续
0,NaN,2188.0,519.0,338.0,0.0,3417.0,70,21,4564.0,0,5,0,14,25,0


In [240]:
df.to_excel('wiki爬取干员信息.xlsx',index=False)